# 서울시 지하철역 위치 표시

# 1 준비작업

## 1.1 Colab 과 GDrive 연동


In [0]:
use_colab = True

In [0]:
if use_colab == True:
  from google.colab import drive
  drive.mount("/content/gdrive")

In [0]:
if use_colab == True:
  work_root_path = "/content/gdrive/My Drive/JK/Gis/"
else:
  work_root_path = "../"    # src 폴더 상위로 올라감
  
input_path = work_root_path+"dataset/"
output_path = work_root_path+"output/"

In [30]:
cd '/content/gdrive/My Drive'

/content/gdrive/My Drive


## 1.2 Import

In [0]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MiniMap

# 2 지도 표시

## 2.1 서울시청 위도, 경도 좌표 파악
구글 맵스 등을 이용하여 서울시청의 위도, 경도를 파악한다.

위도(latitude) : 37.565954

경도(longitude) : 126.978023

## 2.2 서울시 지도 표시

In [32]:
# 서울시청 위도, 경도
latitude_value = 37.565954
longitude_value = 126.978023

map_hs = folium.Map((latitude_value, longitude_value), zoom_start = 11)
mc = MarkerCluster()

mc.add_child(folium.Marker(location = [latitude_value, longitude_value],
                           popup = '서울시청',
                           icon = folium.Icon(color = 'red', icon = 'info-sign')))

minimap = MiniMap()
map_hs.add_child(minimap)

map_hs.add_child(mc)

## 2.3 지하철역 데이타 가져오기

서울 열린데이터 광장에서 서울시 지하철역 좌표 데이타를 가져와서 seoul_subway_stations.csv 파일을 만든다.

http://data.seoul.go.kr/dataList/OA-118/S/1/datasetView.do

이때, 컬럼명은 프로그래밍상의 편의를 위해서 sub_station_code,	station_name,	line_no,	external_code,	cyber_station,	ref_y, 	ref_x,	y_wgs,	x_wgs 로 변경한다.

In [33]:
input_file = input_path+'seoul_subway_stations.csv'
print(input_file)

# utf-8 오류 발생하여 CP949 로 교체함
#subway_stations = pd.read_csv(input_file, encoding = 'utf-8')
subway_stations = pd.read_csv(input_file, encoding ='CP949')

/content/gdrive/My Drive/JK/Gis/dataset/seoul_subway_stations.csv


In [34]:
subway_stations.head()

,sub_station_code,station_name,line_no,external_code,cyber_station,y_ref,x_ref,y_wgs,x_wgs
0,2818,가락시장,8,817,2818,525992.0,1108579.0,37.492522,127.118234
1,340,가락시장,3,350,2818,525992.0,1108579.0,37.492522,127.118234
2,2535,종로3가,5,534,153,498060.0,1130332.0,37.571607,126.991806
3,319,종로3가,3,329,153,498060.0,1130332.0,37.571607,126.991806
4,153,종로3가,1,130,153,498060.0,1130332.0,37.571607,126.991806


## 2.4 지도 표시용 데이타 변환

지도에 표시하기 위해서 필요한 위도, 경도, 지하철역 이름을 추출한다.

In [0]:
loc_data = subway_stations[['y_wgs','x_wgs','station_name']].drop_duplicates(keep = 'first')
loc_data = subway_stations.groupby(['station_name'])['y_wgs','x_wgs'].mean()

In [0]:
loc_data = loc_data.reset_index()

In [0]:
loc_data = loc_data.dropna()

In [38]:
loc_data.head()

,station_name,y_wgs,x_wgs
1,가능,37.748577,127.044213
2,가락시장,37.492522,127.118234
3,가산디지털단지,37.481072,126.882343
4,가양,37.561391,126.854456
6,가재울,37.484192,126.683673


In [39]:
loc_data.shape

(577, 3)

## 2.5 지도에 지하철역 표시

itertuples 를 사용하여 모든 지하철역을 표시한다.

In [40]:
# 서울시청 위도, 경도
latitude_value = 37.565954
longitude_value = 126.978023

map_hs = folium.Map((latitude_value, longitude_value), zoom_start = 11)
mc = MarkerCluster()

idx = 0
for row in loc_data.itertuples():
    mc.add_child(folium.Marker(location = [row.y_wgs, row.x_wgs], 
                               popup = row.station_name))

minimap = MiniMap()
map_hs.add_child(minimap)

map_hs.add_child(mc)

지하철역이 1개이면 파란색으로 표시되고, 여러개가 모여 있으면 숫자로 표시된다.

## 2.6 지표 표시 내용을 html 로 저장

Folium 을 사용할 때 하얗게 출력되면 표시할 것이 많은 경우이므로 html로 저장해서 확인하면 된다.

In [0]:
map_hs.save(output_path+'subway_stations.html')